In [61]:
import os
import xlcalculator


from series_extraction.excel_loader import ExcelLoader
from series_extraction.excel_validator import ExcelValidator
from series_extraction.excel_cleaner import ExcelCleaner
from series_extraction.table_finder import TableFinder
from series_extraction.series_extractor import SeriesExtractor
from series_extraction.excel_compatibility_checker import ExcelCompatibilityChecker
from series_extraction.series_iterator import SeriesIterator
from series_extraction.series_mapper import SeriesMapper

from ast_building.formula_parser import FormulaParser
from ast_building.series_implementer import SeriesImplementer

from formula_visualiser import FormulaVisualiser


In [62]:
current_directory = os.getcwd()

parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
data_directory = os.path.join(parent_directory, 'data')

project_name = 'test_excel_7'

excel_raw_file_path = os.path.join(data_directory, "excel_files_raw", f"{project_name}_raw.xlsx")
excel_reduced_filepath = os.path.join(data_directory, "excel_files_reduced", f"{project_name}_reduced.xlsx")

In [63]:
excel_raw= ExcelLoader.load_file(excel_raw_file_path)
excel_reduced = ExcelLoader.load_file(excel_reduced_filepath)

In [64]:
is_valid = ExcelValidator.validate_excel(excel_reduced)

In [65]:
if not is_valid:
    raise Exception("Excel file is not valid")

In [66]:
excel_reduced_clean = ExcelCleaner.clean_excel(excel_reduced)

In [67]:
extracted_tables, data = TableFinder.find_tables(excel_reduced_clean)

In [68]:
series_data = SeriesExtractor.extract_table_details(extracted_tables, data)

In [69]:
is_compatible = ExcelCompatibilityChecker.check_file(excel_raw, excel_reduced, extracted_tables)

In [70]:
if not is_compatible:
    raise Exception("Excel file is not compatible")

In [71]:
series_dict = SeriesExtractor.extract_series(extracted_tables=extracted_tables, data=data)

In [72]:
series_mapping = SeriesMapper.map_series(series_dict)

In [73]:
series_iterator = SeriesIterator.iterate_series(series_dict)

In [74]:
series_list = [series for series in series_iterator if series.formulas != [None, None]]

In [75]:
for series in series_list:
    formula_1 = series.formulas[0]
    print(formula_1)
    formula_1_ast = FormulaParser.parse_formula(formula_1)
    series_implementer = SeriesImplementer(series_mapping, sheet_name = series.worksheet.sheet_name)
    formula_1_ast_new = series_implementer.replace_range_nodes(formula_1_ast)

=SUM(A2,B2)
cell_range: A2
worksheet: Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None)
cell: [Cell(column=1, row=2, coordinate=None, value=None, value_type=None)]
cell_range: B2
worksheet: Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None)
cell: [Cell(column=2, row=2, coordinate=None, value=None, value_type=None)]
=IF(E2="good","Yes","No")
cell_range: E2
worksheet: Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None)
cell: [Cell(column=5, row=2, coordinate=None, value=None, value_type=None)]
=COUNTIF(B:B,"<0")
cell_range: B:B
worksheet: Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None)
cell: [Cell(column=2, row=1, coordinate=None, value=None, value_type=None), Cell(column=2, row=2, coordinate=None, value=None, value_type=None), Cell(column=2, row=3, coordinate=None, value=None, value_type=None)]
=SUMIF(D:D,">1000")
cell_range: D:D
worksheet: Worksheet(sheet_name='Sheet1', workbook_file_path=None, works

AttributeError: 'NoneType' object has no attribute 'startswith'